In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#import and clean datasets
advanced = pd.read_csv("advanced.csv", header = 1, index_col=1).iloc[1:].dropna(axis=1, how='all')
per_game = pd.read_csv('per_game.csv', index_col=1)
standings = pd.read_csv('standings.csv', header = 1, index_col=1)

#remove astericks in team names
per_game.index = per_game.index.str.replace(r'\*', '', regex=True)
advanced.index = advanced.index.str.replace(r'\*', '', regex=True)

#merge datasets
##TODO: merge datasets for all three
##TODO: rename columns in standings dataset. Add conference/division labels to colums
merged = pd.concat([standings, per_game, advanced], axis=1, join='outer')

In [3]:
try:
    import sklearn
    print(sklearn.__version__)
    if (sklearn.__version__ < '1.3.0'): #update for latest scikit-learn version
        !pip install -U --user scikit-learn
except Exception as e:
    !pip install scikit-learn

1.4.1.post1


In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import PolynomialFeatures

In [5]:
data = merged

# Step 2: Prepare the data
X = data[['3P%']]  # Predictor variable (reshape for a single feature)
y = data['PTS']  # Response variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
new_3P = np.array([[0.35]])  # Example 3P% value
predicted_pts = model.predict(new_3P)
print(f"Predicted PTS for 3P% = 0.35: {predicted_pts[0]}")

RMSE: 2.4531407664302716
Coefficient: 72.91551246537384
Intercept: 88.91222299168982
Predicted PTS for 3P% = 0.35: 114.43265235457065


/Users/edmondniu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [15]:
merged = merged.drop(merged.index[-1])
merged[['Win', 'Loss']] = merged['Overall'].str.split('-', expand=True).astype(int)
merged['WinLossPercentage'] = merged['Win']/merged['Loss']
merged.head(5)

,Rk,Overall,Home,Road,E,W,A,C,SE,NW,...,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G,Win,Loss,WinLossPercentage
Team,,,,,,,,,,,,,,,,,,,,,
Milwaukee Bucks,1.0,58-24,32-9,26-15,35-17,23-7,6-Dec,5-Nov,6-Dec,1-Sep,...,0.520,10.4,77.8,0.175,Fiserv Forum,"718,786","17,531",58,24,2.416667
Boston Celtics,2.0,57-25,32-9,25-16,34-18,23-7,5-Nov,7-Nov,6-Dec,3-Jul,...,0.528,11.3,78.5,0.180,TD Garden,"766,240","18,689",57,25,2.280000
Philadelphia 76ers,3.0,54-28,29-12,25-16,34-18,20-10,6-Oct,13-5,7-Nov,3-Jul,...,0.541,13.0,77.2,0.217,Wells Fargo Center,"839,261","20,470",54,28,1.928571
Denver Nuggets,4.0,53-29,34-7,19-22,19-11,34-18,6-Apr,3-Jul,2-Aug,6-Oct,...,0.543,12.2,76.4,0.201,Ball Arena,"788,635","19,235",53,29,1.827586
Cleveland Cavaliers,5.0,51-31,31-10,20-21,34-18,17-13,10-Aug,13-3,13-5,6-Apr,...,0.535,14.4,76.3,0.210,Rocket Mortgage Fieldhouse,"777,280","18,958",51,31,1.645161


### Defensive Rating only R-squared: 0.41 ###

In [20]:
data = merged

# Step 2: Prepare the data
X = data[['DRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
off_def = np.array([[111.5]])  # Example 3P% value
predicted_pts = model.predict(off_def)
print(f"Predicted Win/Loss Percent for defense: 111.5: {predicted_pts[0]}")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
RMSE: 0.3890913133639424
R-squared: 0.4122736049618033
Coefficient: -0.12626775528890244
Intercept: 15.611750041515744
Predicted Win/Loss Percent for defense: 111.5: 1.5328953268031231


/Users/edmondniu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


### Offensive Rating only: R-squared: -0.099 ###

In [28]:
data = merged

# Step 2: Prepare the data
X = data[['ORtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(3, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
off_def = np.array([[118.0]])  # Example 3P% value
predicted_pts = model.predict(off_def)
print(f"Predicted Win/Loss Percent for offensive: 118.0: {predicted_pts[0]}")

RMSE: 0.5430525843713571
R-squared: -0.14486926092265717
Coefficient: -46.55034121785212
Intercept: 1793.0556960423207


ValueError: X has 1 features, but LinearRegression is expecting 3 features as input.

### Offensive and Defensive Rating R-squared: 0.44 ###

In [37]:
data = merged

# Step 2: Prepare the data
X = data[['ORtg', 'DRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
predicted_pts = model.predict(off_def)
print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3786456121424626
R-squared: 0.44340668972289266
Coefficient: 0.13612600843346287
Intercept: 0.28716517897835336
Predicted Win/Loss Percent for offense: 118, defense: 111.5: 1.9109474052358095


### SRS R-squared: 0.49 ###

In [38]:
data = merged

# Step 2: Prepare the data
X = data[['SRS']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3604944627779787
R-squared: 0.49549053085247285
Coefficient: 0.1384155606339398
Intercept: 1.0492121768942986


### NRtg: R-squared: 0.45 ###

In [39]:
data = merged

# Step 2: Prepare the data
X = data[['NRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3749670686615249
R-squared: 0.45416876973958153
Coefficient: 0.13313244420983103
Intercept: 1.0478406747679347


### TSpercent: R-squared: -0.178 ###

In [40]:
data = merged

# Step 2: Prepare the data
X = data[['TS%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.5509285342103977
R-squared: -0.1783183960695338
Coefficient: 15.422539804270015
Intercept: -7.838438098920609
